In [1]:
import os
import time
import numpy as np

from model import model_load
from model import model_predict

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from cslib import fetch_ts, engineer_features

In [9]:
data_dir = os.path.join("cs-train")


# fetch_ts (cslib)
#   read in new data
#   clean=True when to re-create the csv files in cs-train\ts-data
#
ts_all = fetch_ts(data_dir, clean=False)

... loading ts data from files


In [16]:
# engineer_features (cslib)
#    for any given day the target becomes the sum of the next days revenue
#    for that day we engineer several features that help predict the summed revenue
#    
#    the 'training' flag will trim data that should not be used for training
#    when set to false all data will be returned
#
X, y, dates = engineer_features(df=ts_all['all'], training=True)
        
## Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [4]:
param_grid_rf = {
    'rf__criterion': ['mse','mae'],
    'rf__n_estimators': [10,15,20,25,50,100]
    }

time_start = time.time()
pipe_rf = Pipeline(steps=[('scaler', StandardScaler()), ('rf', RandomForestRegressor())])

# GridSearchCV helps to loop through predefined hyperparameters and fit the model on a training set.
# So, in the end, we can select the best parameters from the listed hyperparameters.
#
grid = GridSearchCV(pipe_rf, param_grid=param_grid_rf, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

rf_mae =  mean_absolute_error(y_test, y_pred)
rf_mse =  mean_squared_error(y_test, y_pred)
rf_r2_score = r2_score(y_test, y_pred)

print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
print("mae = {:.0f}".format(rf_mae))
print("mse = {:.0f}".format(rf_mse))
print("r2_score = {:.3f}".format(rf_r2_score))
print("best params =", grid.best_params_)

train time =  00:00:06
mae = 11078
mse = 238613940
r2_score = 0.964
best params = {'rf__criterion': 'mse', 'rf__n_estimators': 100}


In [5]:
param_grid_gb = {
    'gb__criterion': ['mse','mae'],
    'gb__n_estimators': [10,15,20,25,50,100]
    }

time_start = time.time()
pipe_gb = Pipeline(steps=[('scaler', StandardScaler()), ('gb', GradientBoostingRegressor())])

# GridSearchCV helps to loop through predefined hyperparameters and fit the model on a training set.
# So, in the end, we can select the best parameters from the listed hyperparameters.
#
grid = GridSearchCV(pipe_gb, param_grid=param_grid_gb, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

gb_mae =  mean_absolute_error(y_test, y_pred)
gb_mse =  mean_squared_error(y_test, y_pred)
gb_r2_score = r2_score(y_test, y_pred)

print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
print("mae = {:.0f}".format(gb_mae))
print("mse = {:.0f}".format(gb_mse))
print("r2_score = {:.3f}".format(gb_r2_score))
print("best params =", grid.best_params_)

train time =  00:00:03
mae = 16392
mse = 466453956
r2_score = 0.929
best params = {'gb__criterion': 'mse', 'gb__n_estimators': 100}


In [6]:
param_grid_dt = {
    'dt__criterion': ['mse','mae'],
    'dt__max_depth': [5,10,20,50],
    'dt__min_samples_leaf': [1,2,3,4,5]
    }

time_start = time.time()
pipe_ts = Pipeline(steps=[('scaler', StandardScaler()), ('dt', DecisionTreeRegressor())])

# GridSearchCV helps to loop through predefined hyperparameters and fit the model on a training set.
# So, in the end, we can select the best parameters from the listed hyperparameters.
#
grid = GridSearchCV(pipe_ts, param_grid=param_grid_dt, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

dt_mae =  mean_absolute_error(y_test, y_pred)
dt_mse =  mean_squared_error(y_test, y_pred)
dt_r2_score = r2_score(y_test, y_pred)

print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
print("mae = {:.0f}".format(dt_mae))
print("mse = {:.0f}".format(dt_mse))
print("r2_score = {:.3f}".format(dt_r2_score))
print("best params =", grid.best_params_)

train time =  00:00:00
mae = 14038
mse = 612359563
r2_score = 0.907
best params = {'dt__criterion': 'mse', 'dt__max_depth': 20, 'dt__min_samples_leaf': 1}
